# Source Estimation

## Import

In [ ]:
import os.path as op

import mne
import os
import mne_bids
import vtk

from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse,
                              write_inverse_operator)

## Directory

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
preprocessed_reports = "03_preprocessed"
forward_solution = "04_fwd"
covariance_reports = "05_covariance"
source_estimates = "06_stc"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> evoked_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, source_estimates)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, source_estimates))

In [ ]:
preprocessed_report_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+preprocessed_reports)
forward_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+forward_solution)
covariance_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+covariance_reports)
source_estimates_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+source_estimates)

## Input Epoch Data

In [ ]:
preprocessed_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_preprocessed_epo.fif'
preprocessed_file = os.path.join(preprocessed_report_path, preprocessed_name)
epochs = mne.read_epochs(preprocessed_file)
epochs

In [ ]:
info = mne.io.read_info(preprocessed_file)

## Input Forward Solution

In [ ]:
fwd_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_fwd.fif'
fwd_file = os.path.join(forward_path, fwd_name)
fwd = mne.read_forward_solution(fwd_file)
fwd

## Input Noise Covariance

In [ ]:
cov_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_cov.fif'
cov_file = os.path.join(covariance_path, cov_name)
noise_cov = mne.read_cov(cov_file)
noise_cov

## Inverse Operator

In [ ]:
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)

## Calculate Source Estimate

##### Note: There are many, many different methods to use for source estimates.

In [ ]:
print('######################## Computing source estimate. ########################')

In [ ]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2

stc = apply_inverse(epochs.average(), inverse_operator, lambda2,
                    method=method, pick_ori=None)

### Show Source Estimate

In [ ]:
brain = stc.plot(subject='fsaverage',
                 surface='pial',
                 hemi='both',
                 time_viewer=True)

## Report Source Estimate

In [ ]:
stc_report_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_stc_report.html'
stc_report_path = os.path.join(source_estimates_path, stc_report_title)


stc_report = mne.Report(title='Source Estimate')
stc_report.add_stc(
    stc=stc, subject='fsaverage',
    title='Source Estimate', n_time_points=6  # few for speed
)
stc_report.save(stc_report_path, overwrite=True)

## Save Source Estimate

In [ ]:
stc_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_source_estimate.stc'

stc_path = os.path.join(source_estimates_path, stc_title)
stc.save(stc_path, overwrite=True)